<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN Item Item RMSE</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, item_knn as knn
from lenskit.metrics.predict import rmse
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # train the algorithm with the training dataset
    # Now we run the recommender:
    preds = batch.predict(fittable, test)
    # add the algorithm name for analyzability
    return preds

In [2]:
def model_trainer(neighbours):
    neighbours=int(neighbours)
    print(neighbours)
    all_recs = []
    test_data = []
    algo_ii = knn.ItemItem(neighbours,aggregate='weighted-average') # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('ItemItem', algo_ii, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)

    metric = rmse(all_recs['prediction'], all_recs['rating'])
    return metric

In [5]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(x):
    return {'loss': model_trainer(x), 'status': STATUS_OK }

average=[[] for _ in range(10)]

for y in range(len(average)):
    best = fmin(objective,
        space=hp.uniform('x', 1, 50),
        algo=tpe.suggest,
        max_evals=10
               )
    average[y].append(best)

34                                                                                                                     
43                                                                                                                     
3                                                                                                                      
15                                                                                                                     
18                                                                                                                     
7                                                                                                                      
20                                                                                                                     
30                                                                                                                     
49                                      

31                                                                                                                     
34                                                                                                                     
39                                                                                                                     
40                                                                                                                     
18                                                                                                                     
22                                                                                                                     
29                                                                                                                     
34                                                                                                                     
100%|███████████████████████████████████

In [7]:
average = [[{'x': 30.681121683225463}],
 [{'x': 26.631980797126403}],
 [{'x': 29.81739029363799}],
 [{'x': 16.010518474823364}],
 [{'x': 48.23354234659683}],
 [{'x': 21.41490353537463}],
 [{'x': 29.22333149552323}],
 [{'x': 25.295814448912985}],
 [{'x': 18.372381042625957}],
 [{'x': 44.02508303793156}]]

In [8]:
flat_list = [item for sublist in average for item in sublist]

In [9]:
rmse = [0.899669844383351,0.8952034302435237,0.8966189931566783,0.8986493065477519,0.8982221027559707,0.8918293135261491,
        0.8938831615144291,0.8993576221463044,0.8986639393933152,0.8929233048249747]
rmse = pd.DataFrame(rmse)

In [10]:
average = pd.DataFrame.from_dict(flat_list)
metric = pd.concat([average,rmse],axis=1)
metric.columns = ['Neighbours','RMSE']
metric

,Neighbours,RMSE
0,30.681122,0.899670
1,26.631981,0.895203
2,29.817390,0.896619
3,16.010518,0.898649
4,48.233542,0.898222
5,21.414904,0.891829
6,29.223331,0.893883
7,25.295814,0.899358
8,18.372381,0.898664
9,44.025083,0.892923


In [11]:
metric.mean()

Neighbours    28.970607
RMSE           0.896502
dtype: float64